In [1]:
"""
Boilerplate to implement training different networks on different datasets
with varying config.

I wish a machine could automate setting up decent baseline models and datasets
"""
#  import json
import os
from os.path import exists
import pampy
from simple_parsing import ArgumentParser, choice
from simplepytorch import datasets as D
from simplepytorch import trainlib as TL
from simplepytorch import metrics
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Dataset, DataLoader
from typing import Union, Optional
import dataclasses as dc
import numpy as np
import torch as T
import torchvision.transforms as tvt

from deepfix.models import get_effnetv2, get_resnet, get_efficientnetv1, get_DeepFixEnd2End, DeepFixMLP
from deepfix.models.ghaarconv import convert_conv2d_to_gHaarConv2d
from deepfix.init_from_distribution import init_from_beta, reset_optimizer
from deepfix import deepfix_strategies as dfs
import pytorch_wavelets as pyw

In [2]:
# !ln -sr /ocean/projects/cie160013p/agaudio/data/CheXpert-v1.0-small ~/store/DeepFix/data/

In [3]:
# import sys
# sys.executable

In [4]:
MODELS = {
    ('effnetv2', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_effnetv2(pretrain, int(in_ch), int(out_ch))),
    ('resnet50', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_resnet('resnet50', pretrain, int(in_ch), int(out_ch))),
    ('resnet18', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_resnet('resnet18', pretrain, int(in_ch), int(out_ch))),
    ('efficientnet-b0', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_efficientnetv1('efficientnet-b0', pretrain, int(in_ch), int(out_ch))),
    ('efficientnet-b1', str, str, str): (
        lambda pretrain, in_ch, out_ch: get_efficientnetv1('efficientnet-b1', pretrain, int(in_ch), int(out_ch))),
    ('waveletres18', str, str, str): lambda pretrain, in_ch, out_ch: R(
        pretrain, int(in_ch), int(out_ch)),
    ('waveletmlp', str, str, str, str, str, str, str): (
        lambda mlp_channels, in_ch, out_ch, wavelet_levels, patch_size, in_ch_mul, mlp_depth: get_DeepFixEnd2End(
            int(in_ch), int(out_ch),
            in_ch_multiplier=int(in_ch_mul), wavelet='db1',
            wavelet_levels=int(wavelet_levels), wavelet_patch_size=int(patch_size),
            mlp_depth=int(mlp_depth), mlp_channels=int(mlp_channels),
            mlp_fix_weights='none', mlp_activation=None)
        ),

    #  ('waveletres18v2', str, str, str): lambda pretrain, in_ch, out_ch: (
        #  DeepFixCompression(levels=8, wavelet='coif1', patch_size=1),
        #  R2(pretrain, int(in_ch), int(out_ch))),
}


class R(T.nn.Module):
    def __init__(self, pretrain, in_ch, out_ch):
        super().__init__()
        self.r = get_resnet('resnet18', pretrain, in_ch, out_ch,)
        self.dwt = pyw.DWT(J=8, wave='coif1', mode='zero')

    @staticmethod
    def wavelet_coefficients_as_tensorimage(approx, detail, normalize=False):
        B,C = approx.shape[:2]
        fixed_dims = approx.shape[:-2] # num images in minibatch, num channels, etc
        output_shape = fixed_dims + (
            detail[0].shape[-2]*2,  # input img height
            detail[0].shape[-1]*2)  # input img width
        im = T.zeros(output_shape, device=approx.device, dtype=approx.dtype)
        if normalize:
            norm11 = lambda x: (x / max(x.min()*-1, x.max()))  # into [-1,+1] preserving sign
            #  approx = norm11(approx)
        im[..., :approx.shape[-2], :approx.shape[-1]] = approx if approx is not None else 0
        for level in detail:
            lh, hl, hh = level.unbind(-3)
            h,w = lh.shape[-2:]
            if normalize:
                lh, hl, hh = [norm11(x) for x in [lh, hl, hh]]
            #  im[:h, :w] = approx
            im[..., 0:h, w:w+w] = lh  # horizontal
            im[..., h:h+h, :w] = hl  # vertical
            im[..., h:h+h, w:w+w] = hh  # diagonal
        return im

    def forward(self, x):
        x = self.wavelet_coefficients_as_tensorimage(*self.dwt(x))
        return self.r(x)


class R2(T.nn.Module):
    def __init__(self, pretrain, in_ch, out_ch):
        super().__init__()
        self.r = get_resnet('resnet18', pretrain, in_ch, out_ch,)

    def forward(self, x):
        B,C,H = x.shape
        x = x.unsqueeze(-1).repeat(1,1,1,H)
        return self.r(x)


class LossCheXpertIdentity(T.nn.Module):
    def __init__(self, N):
        super().__init__()
        self.bce = T.nn.BCEWithLogitsLoss()
        self.N = N

    def forward(self, yhat, y):
        # absolute max possible num patients in chexpert is 223414
        # but let's just hash them into a smaller number of bins via modulo N
        assert self.N == yhat.shape[1], \
                f'note: model must have {self.N} binary predictions per sample'
        y_onehot = y.new_zeros(y.shape[0], self.N, dtype=T.float
                               ).scatter_(1, y.long()%self.N, 1)
        return self.bce(yhat[:, -1], y_onehot[:, -1])


class LossCheXpertUignore(T.nn.Module):
    def __init__(self):
        super().__init__()
        self.bce = T.nn.BCEWithLogitsLoss()

    def forward(self, yhat, y):
        ignore = (y != 2)  # ignore uncertainty labels
        return self.bce(yhat[ignore], y[ignore])


def loss_intelmobileodt(yhat, y):
    """BCE Loss with class balancing weights.

    Not sure this actually helps

    because Type 2 is the hardest class, it
    has the most samples, and it separates Type 1 from Type 3.  Arguably, Type 2
    samples are on the decision boundary between Type 1 and 3.
    Class balancing weights make it harder to focus on class 2.
    """
    #  assert y.shape == yhat.shape, 'sanity check'
    #  assert y.dtype == yhat.dtype, 'sanity check'

    # class distribution of stage='train'
    w = T.tensor([249, 781, 450], dtype=y.dtype, device=y.device)
    w = (w.max() / w).reshape(1, 3)
    # w can have any of the shapes:  (B,1) or (1,C) or (B,C)
    #  return T.nn.functional.binary_cross_entropy_with_logits(yhat, y, weight=w)
    return T.nn.functional.cross_entropy(yhat, y, weight=w)
    # can't apply focal loss unless do it manually.


def onehot(y, nclasses):
    return T.zeros((y.numel(), nclasses), dtype=y.dtype, device=y.device)\
            .scatter_(1, y.unsqueeze(1), 1)


def _upsample_pad_minibatch_imgs_to_same_size(batch, target_is_segmentation_mask=False):
    """a collate function for a dataloader of (x,y) samples.  """
    shapes = [item[0].shape for item in batch]
    H = max(h for c,h,w in shapes)
    W = max(w for c,h,w in shapes)
    X, Y = [], []
    for item in batch:
        h,w = item[0].shape[1:]
        dh, dw = (H-h), (W-w)
        padding = (dw//2, dw-dw//2, dh//2, dh-dh//2, )
        X.append(T.nn.functional.pad(item[0], padding))
        if target_is_segmentation_mask:
            Y.append(T.nn.functional.pad(item[1], padding))
        else:
            Y.append(item[1])
    return T.stack(X), T.stack(Y)


def get_dset_chexpert(train_frac=.8, val_frac=.2, small=False,
                      labels:str='diagnostic', num_identities=None):
    """
    Args:
        labels:  either "diagnostic" (the 14 classes defined as
            D.CheXpert.LABELS_DIAGNOSTIC) or "identity" ("patient", "study",
            "view", "index")
        small:  whether to use CheXpert_Small dataset (previously downsampled
            images) or the fully size dataset.
        num_identities:  used only if labels='identity'.  If
            num_identities=1000, then all patients get identified as coming
            from precisely 1 of 1000 bins.

    Returns:
        (
        {'train_dset': ..., 'val_dset': ..., 'test_dset': ...,
         'train_loader': ..., 'val_loader': ..., 'test_loader': ...
         },

        ('Pneumonia', 'Cardiomegaly', ...)  # class names defined by `labels`
        )
    """
    _label_cleanup_dct = dict(D.CheXpert.LABEL_CLEANUP_DICT)
    if labels == 'diagnostic':
        class_names = D.CheXpert.LABELS_DIAGNOSTIC
        for k in class_names:
            _label_cleanup_dct[k][np.nan] = 0  # remap missing value to negative
        get_ylabels = lambda dct: \
                D.CheXpert.format_labels(dct, labels=class_names).float()
    elif labels == 'identity':
        class_names = list(range(num_identities))
        get_ylabels = lambda dct: \
                (D.CheXpert.format_labels(dct, labels=['index']) % num_identities).long()
    else:
        raise NotImplementedError(f"unrecognized labels: {labels}")
    kws = dict(
        img_transform=tvt.Compose([
            #  tvt.RandomCrop((512, 512)),
            tvt.ToTensor(),  # full res 1024x1024 imgs
        ]),
        getitem_transform=lambda dct: (dct['image'], get_ylabels(dct)),
        label_cleanup_dct=_label_cleanup_dct,
    )
    if small:
        kls = D.CheXpert_Small
    else:
        kls = D.CheXpert

    train_dset = kls(dataset_dir="../data/CheXpert-v1.0-small/",use_train_set=True, **kws)  #Edited Elvin

    N = len(train_dset)
    if train_frac + val_frac == 1:
        nsplits = [N - int(N*val_frac), int(N*val_frac), 0]
    else:
        a,b = int(N*train_frac), int(N*val_frac)
        nsplits = [a,b, N-a-b]
    train_dset, val_dset, _ = T.utils.data.random_split(train_dset, nsplits)
    test_dset = kls(dataset_dir="../data/CheXpert-v1.0-small/",use_train_set=False, **kws) #Edited Elvin
    batch_dct = dict(
        batch_size=15, collate_fn=_upsample_pad_minibatch_imgs_to_same_size,
        num_workers=int(os.environ.get("num_workers", 4)))  # upsample pad must take time
    train_loader=DataLoader(train_dset, shuffle=True, **batch_dct)
    val_loader=DataLoader(val_dset, **batch_dct)
    test_loader=DataLoader(test_dset, **batch_dct)
    return (dict(
        train_dset=train_dset, val_dset=val_dset, test_dset=test_dset,
        train_loader=train_loader, val_loader=val_loader, test_loader=test_loader,
    ), class_names)


def get_dset_intel_mobileodt(stage_trainval:str, use_val:str, stage_test:str, augment:str
                             ) -> (dict[str,Optional[Union[Dataset,DataLoader]]], tuple[str]):
    """Obtain train/val/test splits for the IntelMobileODT Cervical Cancer
    Colposcopy dataset, and the data loaders.

    Args:
        stage_trainval: the `stage` for training and validation.
            i.e. Possible choices:  {'train', 'train+additional'}
            Train / val split is 70/30 random stratified split.
        use_val: Whether to create a validation set
            Choices:  {"val", "noval"}
        stage_test: the `stage` for test set.  Should be "test".
        augment: Type of augmentations to apply.  One of {'v1', }.
            "v1" - make the aspect ratio .75, resize images to (200,150), and convert in range [0,1]
    Returns:
        (
        {'train_dset': ..., 'val_dset': ..., 'test_dset': ...,
         'train_loader': ..., 'val_loader': ..., 'test_loader': ...
         },

        ('Type 1', 'Type 2', 'Type 3')
        )
    """
    assert augment == 'v1', 'code bug: other augmentations not implemented'
    base_dir = './data/intel_mobileodt_cervical_resized'
    dset_trainval = D.IntelMobileODTCervical(stage_trainval, base_dir)
    _y = [dset_trainval.getitem(i, load_img=False)
          for i in range(len(dset_trainval))]
    dct = {'test_dset': D.IntelMobileODTCervical(stage_test, base_dir)}
    if use_val == 'noval':
        dct['train_dset'] = dset_trainval
        dct['val_dset'] = None
    else:
        assert use_val == 'val', f'unrecognized option: {use_val}'
        idxs_train, idxs_val = list(
            StratifiedShuffleSplit(1, test_size=.3).split(
                np.arange(len(dset_trainval)), _y))[0]
        dct['train_dset'] = T.utils.data.Subset(dset_trainval, idxs_train)
        dct['val_dset'] = T.utils.data.Subset(dset_trainval, idxs_val)

    # preprocess train/val/test images all the same way
    preprocess_v1 = tvt.Compose([
        #
        ### previously done (to save computation time) ###
        #  D.IntelMobileODTCervical.fix_aspect_ratio,
        #  tvt.Resize((200, 150)),  # interpolation=tvt.InterpolationMode.NEAREST),
        #
        lambda x: x.float()/255.
    ])
    dct = {k: D.PreProcess(v, lambda xy: (
        preprocess_v1(xy[0]),
        #  onehot(xy[1].unsqueeze(0).long()-1, 3).squeeze_().float()))
        xy[1].long()-1))
        for k,v in dct.items()}
    dct.update(dict(
        train_loader=DataLoader(dct['train_dset'], batch_size=20, shuffle=True),
        test_loader=DataLoader(dct['test_dset'], batch_size=20),))
    if dct['val_dset'] is None:
        dct['val_loader'] = None
    else:
        dct['val_loader'] = DataLoader(dct['val_dset'], batch_size=20)
    class_names = [x.replace('_', ' ') for x in D.IntelMobileODTCervical.LABEL_NAMES]
    return dct, class_names


LOSS_FNS = {
    ('BCEWithLogitsLoss', ): lambda _: T.nn.BCEWithLogitsLoss(),
    ('CrossEntropyLoss', ): lambda _: T.nn.CrossEntropyLoss(),
    ('CE_intelmobileodt', ): lambda _: loss_intelmobileodt,
    ('chexpert_uignore', ): lambda _: LossCheXpertUignore(),
    ('chexpert_identity', str): lambda out_ch: LossCheXpertIdentity(N=int(out_ch)),
}

DSETS = {
    ('intel_mobileodt', str, str, str, str): (
        lambda train, val, test, aug: get_dset_intel_mobileodt(train, val, test, aug)),
    #  ('origa', ... todo): ( lambda ...: get_dset_origa(...)
    #  ('riga', ... todo): ( lambda ...: get_dset_riga(...)
    ('chexpert', str, str): (
        lambda train_frac, val_frac: get_dset_chexpert(
            float(train_frac), float(val_frac), small=False, labels='diagnostic')),
    ('chexpert_small', str, str): (
        lambda train_frac, val_frac: get_dset_chexpert(
            float(train_frac), float(val_frac), small=True, labels='diagnostic')),
    ('chexpert_small_ID', str, str, str): (
        lambda num_identities, train_frac, val_frac: get_dset_chexpert(
            float(train_frac), float(val_frac), small=True,
            labels='identity', num_identities=int(num_identities))),
}


def match(spec:str, dct:dict):
    return pampy.match(spec.split(':'), *(x for y in dct.items() for x in y))


def get_model_opt_loss(
        model_spec:str, opt_spec:str, loss_spec:str, regularizer_spec:str,
        device:str) -> dict[str, Union[T.nn.Module, T.optim.Optimizer]]:
    """
    Args:
        model_spec: a string of form,
            "model_name:pretraining:in_channels:out_classes".  For example:
            "effnetv2:untrained:1:5"
        opt_spec: Specifies how to create optimizer.
            First value is a pytorch Optimizer in T.optim.*.
            Other values are numerical parameters.
            Example: "SGD:lr=.003:momentum=.9"
        device: e.g. 'cpu' or 'gpu'
    Returns:
        a pytorch model and optimizer
    """
    mdl = match(model_spec, MODELS)
    mdl = mdl.to(device, non_blocking=True)
    optimizer = reset_optimizer(opt_spec, mdl)
    loss_fn = match(loss_spec, LOSS_FNS)
    if regularizer_spec != 'none':
        loss_fn = RegularizedLoss(mdl, loss_fn, regularizer_spec)
    return dict(model=mdl, optimizer=optimizer, loss_fn=loss_fn)


class RegularizedLoss(T.nn.Module):
    def __init__(self, model, lossfn, regularizer_spec:str):
        super().__init__()
        self.lossfn = lossfn
        self.regularizer_spec = regularizer_spec
        if regularizer_spec == 'none':
            self.regularizer = lambda *y: 0
        elif regularizer_spec.startswith('deepfixmlp:'):
            lbda = float(regularizer_spec.split(':')[1])
            self.regularizer = lambda *y: (
                float(lbda) * DeepFixMLP.get_VecAttn_regularizer(model))
        else:
            raise NotImplementedError(regularizer_spec)

    def forward(self, yhat, y):
        return self.lossfn(yhat, y) + self.regularizer(yhat, y)

    def __repr__(self):
        return f'RegularizedLoss<{repr(self.lossfn)},{self.regularizer_spec}>'


def get_dset_loaders_resultfactory(dset_spec:str) -> dict:
    dct, class_names = match(dset_spec, DSETS)
    if any(dset_spec.startswith(x) for x in {'intel_mobileodt:',
                                             'chexpert_small_ID:'}):
        #  dct['result_factory'] = lambda: TL.MultiLabelBinaryClassification(
                #  class_names, binarize_fn=lambda yh: (T.sigmoid(yh)>.5).long())
        dct['result_factory'] = lambda: TL.MultiClassClassification(
                len(class_names), binarize_fn=lambda yh: yh.softmax(1).argmax(1))
    elif any(dset_spec.startswith(x) for x in {'chexpert:', 'chexpert_small:'}):
        dct['result_factory'] = lambda: CheXpertMultiLabelBinaryClassification(
            class_names, binarize_fn=lambda yh: (yh.sigmoid()>.5).long(), report_avg=True)
    else:
        raise NotImplementedError(f"I don't know how to create the result factory for {dset_spec}")
    return dct

class CheXpertMultiLabelBinaryClassification(TL.MultiLabelBinaryClassification):
    def update(self, yhat, y, loss) -> None:
        self.num_samples += yhat.shape[0]
        self.loss += loss.item()
        assert yhat.shape == y.shape
        assert yhat.ndim == 2 and yhat.shape[1] == len(self._cms), "sanity check: model outputs expected prediction shape"
        binarized = self._binarize_fn(yhat)
        assert binarized.dtype == T.long, 'sanity check binarize fn'
        assert binarized.shape == y.shape, 'sanity check binarize fn'
        ignore = (y != 2)  # ignore uncertainty labels
        for i, (kls, cm) in enumerate(self._cms.items()):
            rows = ignore[:, i]
            if rows.sum() == 0:
                continue  # don't update a confusion matrix if all data for this class is ignored
            cm += metrics.confusion_matrix(y[rows, i], binarized[rows, i], num_classes=2).cpu()


def get_deepfix_train_strategy(args:'TrainOptions'):
    deepfix_spec = args.deepfix
    if deepfix_spec == 'off':
        return TL.train_one_epoch
    elif deepfix_spec.startswith('reinit:'):
        _, N, P, R = deepfix_spec.split(':')
        return dfs.DeepFix_TrainOneEpoch(int(N), float(P), int(R), TL.train_one_epoch)
    elif deepfix_spec.startswith('dhist:'):
        fp = deepfix_spec.split(':', 1)[1]
        assert exists(fp), f'histogram file not found: {fp}'
        return dfs.DeepFix_DHist(fp)
    elif deepfix_spec.startswith('dfhist:'):
        fp = deepfix_spec.split(':', 1)[1]
        assert exists(fp), f'histogram file not found: {fp}'
        return dfs.DeepFix_DHist(fp, fixed=True)
    elif deepfix_spec == 'fixed':
        return dfs.DeepFix_DHist('', fixed=True, init_with_hist=False)
    elif deepfix_spec.startswith('beta:'):
        alpha, beta = deepfix_spec.split(':')[1:]
        return dfs.DeepFix_LambdaInit(
            lambda cfg: init_from_beta(cfg.model, float(alpha), float(beta)))
    elif deepfix_spec.startswith('ghaarconv2d:'):
        ignore_layers = deepfix_spec.split(':')[1].split(',')
        return dfs.DeepFix_LambdaInit(
            lambda cfg: (
                print(f'initialize {deepfix_spec}'),
                convert_conv2d_to_gHaarConv2d(cfg.model, ignore_layers=ignore_layers),
                reset_optimizer(args.opt, cfg.model),
                print(cfg.model)
            ))
    else:
        raise NotImplementedError(deepfix_spec)


def train_config(args:'TrainOptions') -> TL.TrainConfig:
    return TL.TrainConfig(
        **get_model_opt_loss(
            args.model, args.opt, args.lossfn, args.loss_reg, args.device),
        **get_dset_loaders_resultfactory(args.dset),
        device=args.device,
        epochs=args.epochs,
        start_epoch=args.start_epoch,
        train_one_epoch=get_deepfix_train_strategy(args),
        experiment_id=args.experiment_id,
    )


@dc.dataclass
class TrainOptions:
    """High-level configuration for training PyTorch models
    on the IntelMobileODTCervical dataset.
    """
    epochs:int = 50
    start_epoch:int = 0  # if "--start_epoch 1", then don't evaluate perf before training.
    device:str = 'cuda' if T.cuda.is_available() else 'cpu'
    dset:str = None #choice(
        #  'intel_mobileodt:train:val:test:v1',
        #  'intel_mobileodt:train+additional:val:test:v1',
        #  'intel_mobileodt:train+additional:noval:test:v1',
        #  'chexpert:.8:.2', 'chexpert:.01:.01', 'chexpert:.001:.001',
        #  'chexpert_small:.8:.2', 'chexpert_small:.01:.01',
        #   'chexpert_small:.001:.001',
        #  default='intel_mobileodt:train:val:test:v1')
    opt:str = 'SGD:lr=.001:momentum=.9:nesterov=1'
    lossfn:str = None  # choices:
        #  'BCEWithLogitsLoss',
        #  'CrossEntropyLoss', 
        #  'CE_intelmobileodt',
        #  'chexpert_uignore', 
        #  'chexpert_identity:N' for some N=num_identities predicted by model (compared to identities y%N)
    loss_reg:str = 'none'  # Optionally add a regularizer to the loss.  loss + reg.  Accepted values:  'none', 'deepfixmlp:X' where X is a positive float denoting the lambda in l1 regularizer
    model:str = 'resnet18:imagenet:3:3'  # Model specification adheres to the template "model_name:pretraining:in_ch:out_ch"
    deepfix:str = 'off'  # DeepFix Re-initialization Method.
                         #  "off" or "reinit:N:P:R" or "d[f]hist:path_to_histogram.pth"
                         #  or "beta:A:B" for A,B as (float) parameters of the beta distribution
                         # 'ghaarconv2d:layer1,layer2' Replaces all spatial convolutions with GHaarConv2d layer except the specified layers
    experiment_id:str = os.environ.get('run_id', 'debugging')
    prune:str = 'off'

    def execute(self):
        cfg = train_config(self)
        cfg.train(cfg)


def main():
    p = ArgumentParser()
    p.add_arguments(TrainOptions, dest='TrainOptions')
    for patch_size in [6]:
#         for wavelet_level in [1,2,3,4,5,6]:
        for wavelet_level in [6,7,8,9]:    
            for loss_reg in ["deepfixmlp:0.75"]:
                try:
                    model_params = "waveletmlp:300:1:14:"+str(wavelet_level)+":"+str(patch_size)+":1:2"
                    print(model_params)
                    exp_id = 'model_'+model_params+'_level_' + str(wavelet_level) + '_patch_size_' + str(patch_size) + '_loss_reg_' + loss_reg 
                    args = p.parse_args(["--dset", "chexpert_small:.01:.01", "--opt", "Adam:lr=0.001", "--lossfn", "chexpert_uignore", "--model", model_params, "--loss_reg", loss_reg,"--experiment_id",exp_id]).TrainOptions

                    print(args)
                    cfg = train_config(args)

                # python deepfix/train.py --dset chexpert_small:.01:.01 --opt Adam:lr=0.001 --lossfn chexpert_uignore --model waveletmlp:300:1:14:7:1:1:2 --loss_reg none    

                    if args.prune != 'off':
                        assert args.prune.startswith('ChannelPrune:')
                        raise NotImplementedError('code is a bit hardcoded, so it is not available without hacking on it.')
                        print(args.prune)
                        from explainfix import channelprune
                        from deepfix.weight_saliency import costfn_multiclass
                        a = sum([x.numel() for x in cfg.model.parameters()])
                        channelprune(cfg.model, pct=5, grad_cost_fn=costfn_multiclass,
                                     loader=cfg.train_loader, device=cfg.device, num_minibatches=10)
                        b = sum([x.numel() for x in cfg.model.parameters()])
                        assert a/b != 1
                        print(f'done channelpruning.  {a/b}')

                    cfg.train(cfg)
                except Exception as e:
                    print("=================================================================================================")
                    print(e)
                    print("=================================================================================================")

            print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    #  import IPython ; IPython.embed() ; import sys ; sys.exit()

    #  with T.profiler.profile(
    #      activities=[
    #          T.profiler.ProfilerActivity.CPU,
    #          T.profiler.ProfilerActivity.CUDA,
    #      ], with_modules=True,
    #  ) as p:
    #      cfg.train(cfg)
    #  print(p.key_averages().table(
    #      sort_by="self_cuda_time_total", row_limit=-1))


if __name__ == "__main__":
    main()


waveletmlp:300:1:14:6:6:1:2
TrainOptions(epochs=50, start_epoch=0, device='cuda', dset='chexpert_small:.01:.01', opt='Adam:lr=0.001', lossfn='chexpert_uignore', loss_reg='deepfixmlp:0.75', model='waveletmlp:300:1:14:6:6:1:2', deepfix='off', experiment_id='model_waveletmlp:300:1:14:6:6:1:2_level_6_patch_size_6_loss_reg_deepfixmlp:0.75', prune='off')
Checkpoint ./results/model_waveletmlp:300:1:14:6:6:1:2_level_6_patch_size_6_loss_reg_deepfixmlp:0.75/checkpoints/epoch_0.pth


100%|██████████| 16/16 [00:03<00:00,  4.65it/s]
INFO:simplepytorch.logging_tools:DataLogger writing to file: /ocean/projects/cie160013p/elvinj/DeepFix/deepfix/results/model_waveletmlp:300:1:14:6:6:1:2_level_6_patch_size_6_loss_reg_deepfixmlp:0.75/log/20220217T225237.646602_perf.csv


epoch:    0, seconds_training_epoch: nan
	TRAIN RESULTS: train_Loss:  154.36299, train_Num Samples:  2234.00000, train_MCC No Finding:  0.02176, train_MCC Enlarged Cardiomediastinum:  0.00375, train_MCC Cardiomegaly:  0.00962, train_MCC Lung Opacity:  0.06565, train_MCC Lung Lesion: -0.00084, train_MCC Edema: -0.01708, train_MCC Consolidation:  0.01849, train_MCC Pneumonia:  0.00708, train_MCC Atelectasis:  nan, train_MCC Pneumothorax: -0.05623, train_MCC Pleural Effusion: -0.01359, train_MCC Pleural Other: -0.05604, train_MCC Fracture: -0.04983, train_MCC Support Devices:  0.00541, train_MCC AVG:  nan, train_Precision No Finding:  0.11283, train_Precision Enlarged Cardiomediastinum:  0.05598, train_Precision Cardiomegaly:  0.12183, train_Precision Lung Opacity:  0.51457, train_Precision Lung Lesion:  0.04000, train_Precision Edema:  0.00000, train_Precision Consolidation:  0.07720, train_Precision Pneumonia:  0.04321, train_Precision Atelectasis:  0.00000, train_Precision Pneumothorax

100%|██████████| 16/16 [00:03<00:00,  4.30it/s]


epoch:    1, seconds_training_epoch: 31.669
	TRAIN RESULTS: train_Loss:  115.11091, train_Num Samples:  2234.00000, train_MCC No Finding:  0.02148, train_MCC Enlarged Cardiomediastinum: -0.00743, train_MCC Cardiomegaly:  0.00239, train_MCC Lung Opacity:  0.06541, train_MCC Lung Lesion: -0.00443, train_MCC Edema:  0.02980, train_MCC Consolidation: -0.01081, train_MCC Pneumonia: -0.00443, train_MCC Atelectasis: -0.01834, train_MCC Pneumothorax:  0.01351, train_MCC Pleural Effusion:  0.07903, train_MCC Pleural Other: -0.00538, train_MCC Fracture: -0.00623, train_MCC Support Devices:  0.16472, train_MCC AVG:  0.02281, train_Precision No Finding:  0.16129, train_Precision Enlarged Cardiomediastinum:  0.00000, train_Precision Cardiomegaly:  0.12500, train_Precision Lung Opacity:  0.52627, train_Precision Lung Lesion:  0.00000, train_Precision Edema:  0.27684, train_Precision Consolidation:  0.04545, train_Precision Pneumonia:  0.00000, train_Precision Atelectasis:  0.16049, train_Precision P

...

Checkpoint ./results/model_waveletmlp:300:1:14:6:6:1:2_level_6_patch_size_6_loss_reg_deepfixmlp:0.75/checkpoints/epoch_50.pth
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
waveletmlp:300:1:14:7:6:1:2
TrainOptions(epochs=50, start_epoch=0, device='cuda', dset='chexpert_small:.01:.01', opt='Adam:lr=0.001', lossfn='chexpert_uignore', loss_reg='deepfixmlp:0.75', model='waveletmlp:300:1:14:7:6:1:2', deepfix='off', experiment_id='model_waveletmlp:300:1:14:7:6:1:2_level_7_patch_size_6_loss_reg_deepfixmlp:0.75', prune='off')
Checkpoint ./results/model_waveletmlp:300:1:14:7:6:1:2_level_7_patch_size_6_loss_reg_deepfixmlp:0.75/checkpoints/epoch_0.pth


  0%|          | 0/149 [00:00<?, ?it/s]/ocean/projects/cie160013p/elvinj/DeepFix/deepfix/models/waveletmlp.py:124: UserWarning: Input data spatial dimensions are too small for choice of wavelet level and patch size.  Decrease the wavelet level or patch size. Patch size = 6, level = 7, input_shape=torch.Size([15, 16384, 4, 4])
  warnings.warn(msg)
  0%|          | 0/149 [00:01<?, ?it/s]


The size of tensor a (32768) must match the size of tensor b (1179648) at non-singleton dimension 2
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
waveletmlp:300:1:14:8:6:1:2
TrainOptions(epochs=50, start_epoch=0, device='cuda', dset='chexpert_small:.01:.01', opt='Adam:lr=0.001', lossfn='chexpert_uignore', loss_reg='deepfixmlp:0.75', model='waveletmlp:300:1:14:8:6:1:2', deepfix='off', experiment_id='model_waveletmlp:300:1:14:8:6:1:2_level_8_patch_size_6_loss_reg_deepfixmlp:0.75', prune='off')
Checkpoint ./results/model_waveletmlp:300:1:14:8:6:1:2_level_8_patch_size_6_loss_reg_deepfixmlp:0.75/checkpoints/epoch_0.pth


  0%|          | 0/149 [00:00<?, ?it/s]/ocean/projects/cie160013p/elvinj/DeepFix/deepfix/models/waveletmlp.py:124: UserWarning: Input data spatial dimensions are too small for choice of wavelet level and patch size.  Decrease the wavelet level or patch size. Patch size = 6, level = 8, input_shape=torch.Size([15, 65536, 2, 2])
  warnings.warn(msg)
  0%|          | 0/149 [00:01<?, ?it/s]


The size of tensor a (131072) must match the size of tensor b (4718592) at non-singleton dimension 2
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
waveletmlp:300:1:14:9:6:1:2
TrainOptions(epochs=50, start_epoch=0, device='cuda', dset='chexpert_small:.01:.01', opt='Adam:lr=0.001', lossfn='chexpert_uignore', loss_reg='deepfixmlp:0.75', model='waveletmlp:300:1:14:9:6:1:2', deepfix='off', experiment_id='model_waveletmlp:300:1:14:9:6:1:2_level_9_patch_size_6_loss_reg_deepfixmlp:0.75', prune='off')
CUDA out of memory. Tried to allocate 21.09 GiB (GPU 0; 31.75 GiB total capacity; 5.36 GiB already allocated; 13.73 GiB free; 5.62 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


In [5]:
str(0.5)

'0.5'

In [ ]:
# MODELS = {
#     ('effnetv2', str, str, str): (
#         lambda pretrain, in_ch, out_ch: get_effnetv2(pretrain, int(in_ch), int(out_ch))),
#     ('resnet50', str, str, str): (
#         lambda pretrain, in_ch, out_ch: get_resnet('resnet50', pretrain, int(in_ch), int(out_ch))),
#     ('resnet18', str, str, str): (
#         lambda pretrain, in_ch, out_ch: get_resnet('resnet18', pretrain, int(in_ch), int(out_ch))),
#     ('efficientnet-b0', str, str, str): (
#         lambda pretrain, in_ch, out_ch: get_efficientnetv1('efficientnet-b0', pretrain, int(in_ch), int(out_ch))),
#     ('efficientnet-b1', str, str, str): (
#         lambda pretrain, in_ch, out_ch: get_efficientnetv1('efficientnet-b1', pretrain, int(in_ch), int(out_ch))),
#     ('waveletres18', str, str, str): lambda pretrain, in_ch, out_ch: R(
#         pretrain, int(in_ch), int(out_ch)),
#     ('waveletmlp', str, str, str, str, str, str, str): (
#         lambda mlp_channels, in_ch, out_ch, wavelet_levels, patch_size, in_ch_mul, mlp_depth: get_DeepFixEnd2End(
#             int(in_ch), int(out_ch),
#             in_ch_multiplier=int(in_ch_mul), wavelet='db1',
#             wavelet_levels=int(wavelet_levels), wavelet_patch_size=int(patch_size),
#             mlp_depth=int(mlp_depth), mlp_channels=int(mlp_channels),
#             mlp_fix_weights='none', mlp_activation=None)
#         ),

#     #  ('waveletres18v2', str, str, str): lambda pretrain, in_ch, out_ch: (
#         #  DeepFixCompression(levels=8, wavelet='coif1', patch_size=1),
#         #  R2(pretrain, int(in_ch), int(out_ch))),
# }


# class R(T.nn.Module):
#     def __init__(self, pretrain, in_ch, out_ch):
#         super().__init__()
#         self.r = get_resnet('resnet18', pretrain, in_ch, out_ch,)
#         self.dwt = pyw.DWT(J=8, wave='coif1', mode='zero')

#     @staticmethod
#     def wavelet_coefficients_as_tensorimage(approx, detail, normalize=False):
#         B,C = approx.shape[:2]
#         fixed_dims = approx.shape[:-2] # num images in minibatch, num channels, etc
#         output_shape = fixed_dims + (
#             detail[0].shape[-2]*2,  # input img height
#             detail[0].shape[-1]*2)  # input img width
#         im = T.zeros(output_shape, device=approx.device, dtype=approx.dtype)
#         if normalize:
#             norm11 = lambda x: (x / max(x.min()*-1, x.max()))  # into [-1,+1] preserving sign
#             #  approx = norm11(approx)
#         im[..., :approx.shape[-2], :approx.shape[-1]] = approx if approx is not None else 0
#         for level in detail:
#             lh, hl, hh = level.unbind(-3)
#             h,w = lh.shape[-2:]
#             if normalize:
#                 lh, hl, hh = [norm11(x) for x in [lh, hl, hh]]
#             #  im[:h, :w] = approx
#             im[..., 0:h, w:w+w] = lh  # horizontal
#             im[..., h:h+h, :w] = hl  # vertical
#             im[..., h:h+h, w:w+w] = hh  # diagonal
#         return im

#     def forward(self, x):
#         x = self.wavelet_coefficients_as_tensorimage(*self.dwt(x))
#         return self.r(x)


# class R2(T.nn.Module):
#     def __init__(self, pretrain, in_ch, out_ch):
#         super().__init__()
#         self.r = get_resnet('resnet18', pretrain, in_ch, out_ch,)

#     def forward(self, x):
#         B,C,H = x.shape
#         x = x.unsqueeze(-1).repeat(1,1,1,H)
#         return self.r(x)


# class LossCheXpertIdentity(T.nn.Module):
#     def __init__(self, N):
#         super().__init__()
#         self.bce = T.nn.BCEWithLogitsLoss()
#         self.N = N

#     def forward(self, yhat, y):
#         # absolute max possible num patients in chexpert is 223414
#         # but let's just hash them into a smaller number of bins via modulo N
#         assert self.N == yhat.shape[1], \
#                 f'note: model must have {self.N} binary predictions per sample'
#         y_onehot = y.new_zeros(y.shape[0], self.N, dtype=T.float
#                                ).scatter_(1, y.long()%self.N, 1)
#         return self.bce(yhat[:, -1], y_onehot[:, -1])


# class LossCheXpertUignore(T.nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.bce = T.nn.BCEWithLogitsLoss()

#     def forward(self, yhat, y):
#         ignore = (y != 2)  # ignore uncertainty labels
#         return self.bce(yhat[ignore], y[ignore])


# def loss_intelmobileodt(yhat, y):
#     """BCE Loss with class balancing weights.

#     Not sure this actually helps

#     because Type 2 is the hardest class, it
#     has the most samples, and it separates Type 1 from Type 3.  Arguably, Type 2
#     samples are on the decision boundary between Type 1 and 3.
#     Class balancing weights make it harder to focus on class 2.
#     """
#     #  assert y.shape == yhat.shape, 'sanity check'
#     #  assert y.dtype == yhat.dtype, 'sanity check'

#     # class distribution of stage='train'
#     w = T.tensor([249, 781, 450], dtype=y.dtype, device=y.device)
#     w = (w.max() / w).reshape(1, 3)
#     # w can have any of the shapes:  (B,1) or (1,C) or (B,C)
#     #  return T.nn.functional.binary_cross_entropy_with_logits(yhat, y, weight=w)
#     return T.nn.functional.cross_entropy(yhat, y, weight=w)
#     # can't apply focal loss unless do it manually.


# def onehot(y, nclasses):
#     return T.zeros((y.numel(), nclasses), dtype=y.dtype, device=y.device)\
#             .scatter_(1, y.unsqueeze(1), 1)


# def _upsample_pad_minibatch_imgs_to_same_size(batch, target_is_segmentation_mask=False):
#     """a collate function for a dataloader of (x,y) samples.  """
#     shapes = [item[0].shape for item in batch]
#     H = max(h for c,h,w in shapes)
#     W = max(w for c,h,w in shapes)
#     X, Y = [], []
#     for item in batch:
#         h,w = item[0].shape[1:]
#         dh, dw = (H-h), (W-w)
#         padding = (dw//2, dw-dw//2, dh//2, dh-dh//2, )
#         X.append(T.nn.functional.pad(item[0], padding))
#         if target_is_segmentation_mask:
#             Y.append(T.nn.functional.pad(item[1], padding))
#         else:
#             Y.append(item[1])
#     return T.stack(X), T.stack(Y)


# def get_dset_chexpert(train_frac=.8, val_frac=.2, small=False,
#                       labels:str='diagnostic', num_identities=None):
#     """
#     Args:
#         labels:  either "diagnostic" (the 14 classes defined as
#             D.CheXpert.LABELS_DIAGNOSTIC) or "identity" ("patient", "study",
#             "view", "index")
#         small:  whether to use CheXpert_Small dataset (previously downsampled
#             images) or the fully size dataset.
#         num_identities:  used only if labels='identity'.  If
#             num_identities=1000, then all patients get identified as coming
#             from precisely 1 of 1000 bins.

#     Returns:
#         (
#         {'train_dset': ..., 'val_dset': ..., 'test_dset': ...,
#          'train_loader': ..., 'val_loader': ..., 'test_loader': ...
#          },

#         ('Pneumonia', 'Cardiomegaly', ...)  # class names defined by `labels`
#         )
#     """
#     _label_cleanup_dct = dict(D.CheXpert.LABEL_CLEANUP_DICT)
#     if labels == 'diagnostic':
#         class_names = D.CheXpert.LABELS_DIAGNOSTIC
#         for k in class_names:
#             _label_cleanup_dct[k][np.nan] = 0  # remap missing value to negative
#         get_ylabels = lambda dct: \
#                 D.CheXpert.format_labels(dct, labels=class_names).float()
#     elif labels == 'identity':
#         class_names = list(range(num_identities))
#         get_ylabels = lambda dct: \
#                 (D.CheXpert.format_labels(dct, labels=['index']) % num_identities).long()
#     else:
#         raise NotImplementedError(f"unrecognized labels: {labels}")
#     kws = dict(
#         img_transform=tvt.Compose([
#             #  tvt.RandomCrop((512, 512)),
#             tvt.ToTensor(),  # full res 1024x1024 imgs
#         ]),
#         getitem_transform=lambda dct: (dct['image'], get_ylabels(dct)),
#         label_cleanup_dct=_label_cleanup_dct,
#     )
#     if small:
#         kls = D.CheXpert_Small
#     else:
#         kls = D.CheXpert

#     train_dset = kls(dataset_dir="../data/CheXpert-v1.0-small/",use_train_set=True, **kws)  #Edited Elvin

#     N = len(train_dset)
#     if train_frac + val_frac == 1:
#         nsplits = [N - int(N*val_frac), int(N*val_frac), 0]
#     else:
#         a,b = int(N*train_frac), int(N*val_frac)
#         nsplits = [a,b, N-a-b]
#     train_dset, val_dset, _ = T.utils.data.random_split(train_dset, nsplits)
#     test_dset = kls(dataset_dir="../data/CheXpert-v1.0-small/",use_train_set=False, **kws) #Edited Elvin
#     batch_dct = dict(
#         batch_size=15, collate_fn=_upsample_pad_minibatch_imgs_to_same_size,
#         num_workers=int(os.environ.get("num_workers", 4)))  # upsample pad must take time
#     train_loader=DataLoader(train_dset, shuffle=True, **batch_dct)
#     val_loader=DataLoader(val_dset, **batch_dct)
#     test_loader=DataLoader(test_dset, **batch_dct)
#     return (dict(
#         train_dset=train_dset, val_dset=val_dset, test_dset=test_dset,
#         train_loader=train_loader, val_loader=val_loader, test_loader=test_loader,
#     ), class_names)


# def get_dset_intel_mobileodt(stage_trainval:str, use_val:str, stage_test:str, augment:str
#                              ) -> (dict[str,Optional[Union[Dataset,DataLoader]]], tuple[str]):
#     """Obtain train/val/test splits for the IntelMobileODT Cervical Cancer
#     Colposcopy dataset, and the data loaders.

#     Args:
#         stage_trainval: the `stage` for training and validation.
#             i.e. Possible choices:  {'train', 'train+additional'}
#             Train / val split is 70/30 random stratified split.
#         use_val: Whether to create a validation set
#             Choices:  {"val", "noval"}
#         stage_test: the `stage` for test set.  Should be "test".
#         augment: Type of augmentations to apply.  One of {'v1', }.
#             "v1" - make the aspect ratio .75, resize images to (200,150), and convert in range [0,1]
#     Returns:
#         (
#         {'train_dset': ..., 'val_dset': ..., 'test_dset': ...,
#          'train_loader': ..., 'val_loader': ..., 'test_loader': ...
#          },

#         ('Type 1', 'Type 2', 'Type 3')
#         )
#     """
#     assert augment == 'v1', 'code bug: other augmentations not implemented'
#     base_dir = './data/intel_mobileodt_cervical_resized'
#     dset_trainval = D.IntelMobileODTCervical(stage_trainval, base_dir)
#     _y = [dset_trainval.getitem(i, load_img=False)
#           for i in range(len(dset_trainval))]
#     dct = {'test_dset': D.IntelMobileODTCervical(stage_test, base_dir)}
#     if use_val == 'noval':
#         dct['train_dset'] = dset_trainval
#         dct['val_dset'] = None
#     else:
#         assert use_val == 'val', f'unrecognized option: {use_val}'
#         idxs_train, idxs_val = list(
#             StratifiedShuffleSplit(1, test_size=.3).split(
#                 np.arange(len(dset_trainval)), _y))[0]
#         dct['train_dset'] = T.utils.data.Subset(dset_trainval, idxs_train)
#         dct['val_dset'] = T.utils.data.Subset(dset_trainval, idxs_val)

#     # preprocess train/val/test images all the same way
#     preprocess_v1 = tvt.Compose([
#         #
#         ### previously done (to save computation time) ###
#         #  D.IntelMobileODTCervical.fix_aspect_ratio,
#         #  tvt.Resize((200, 150)),  # interpolation=tvt.InterpolationMode.NEAREST),
#         #
#         lambda x: x.float()/255.
#     ])
#     dct = {k: D.PreProcess(v, lambda xy: (
#         preprocess_v1(xy[0]),
#         #  onehot(xy[1].unsqueeze(0).long()-1, 3).squeeze_().float()))
#         xy[1].long()-1))
#         for k,v in dct.items()}
#     dct.update(dict(
#         train_loader=DataLoader(dct['train_dset'], batch_size=20, shuffle=True),
#         test_loader=DataLoader(dct['test_dset'], batch_size=20),))
#     if dct['val_dset'] is None:
#         dct['val_loader'] = None
#     else:
#         dct['val_loader'] = DataLoader(dct['val_dset'], batch_size=20)
#     class_names = [x.replace('_', ' ') for x in D.IntelMobileODTCervical.LABEL_NAMES]
#     return dct, class_names


# LOSS_FNS = {
#     ('BCEWithLogitsLoss', ): lambda _: T.nn.BCEWithLogitsLoss(),
#     ('CrossEntropyLoss', ): lambda _: T.nn.CrossEntropyLoss(),
#     ('CE_intelmobileodt', ): lambda _: loss_intelmobileodt,
#     ('chexpert_uignore', ): lambda _: LossCheXpertUignore(),
#     ('chexpert_identity', str): lambda out_ch: LossCheXpertIdentity(N=int(out_ch)),
# }

# DSETS = {
#     ('intel_mobileodt', str, str, str, str): (
#         lambda train, val, test, aug: get_dset_intel_mobileodt(train, val, test, aug)),
#     #  ('origa', ... todo): ( lambda ...: get_dset_origa(...)
#     #  ('riga', ... todo): ( lambda ...: get_dset_riga(...)
#     ('chexpert', str, str): (
#         lambda train_frac, val_frac: get_dset_chexpert(
#             float(train_frac), float(val_frac), small=False, labels='diagnostic')),
#     ('chexpert_small', str, str): (
#         lambda train_frac, val_frac: get_dset_chexpert(
#             float(train_frac), float(val_frac), small=True, labels='diagnostic')),
#     ('chexpert_small_ID', str, str, str): (
#         lambda num_identities, train_frac, val_frac: get_dset_chexpert(
#             float(train_frac), float(val_frac), small=True,
#             labels='identity', num_identities=int(num_identities))),
# }


# def match(spec:str, dct:dict):
#     return pampy.match(spec.split(':'), *(x for y in dct.items() for x in y))


# def get_model_opt_loss(
#         model_spec:str, opt_spec:str, loss_spec:str, regularizer_spec:str,
#         device:str) -> dict[str, Union[T.nn.Module, T.optim.Optimizer]]:
#     """
#     Args:
#         model_spec: a string of form,
#             "model_name:pretraining:in_channels:out_classes".  For example:
#             "effnetv2:untrained:1:5"
#         opt_spec: Specifies how to create optimizer.
#             First value is a pytorch Optimizer in T.optim.*.
#             Other values are numerical parameters.
#             Example: "SGD:lr=.003:momentum=.9"
#         device: e.g. 'cpu' or 'gpu'
#     Returns:
#         a pytorch model and optimizer
#     """
#     mdl = match(model_spec, MODELS)
#     mdl = mdl.to(device, non_blocking=True)
#     optimizer = reset_optimizer(opt_spec, mdl)
#     loss_fn = match(loss_spec, LOSS_FNS)
#     if regularizer_spec != 'none':
#         loss_fn = RegularizedLoss(mdl, loss_fn, regularizer_spec)
#     return dict(model=mdl, optimizer=optimizer, loss_fn=loss_fn)


# class RegularizedLoss(T.nn.Module):
#     def __init__(self, model, lossfn, regularizer_spec:str):
#         super().__init__()
#         self.lossfn = lossfn
#         self.regularizer_spec = regularizer_spec
#         if regularizer_spec == 'none':
#             self.regularizer = lambda *y: 0
#         elif regularizer_spec.startswith('deepfixmlp:'):
#             lbda = float(regularizer_spec.split(':')[1])
#             self.regularizer = lambda *y: (
#                 float(lbda) * DeepFixMLP.get_VecAttn_regularizer(model))
#         else:
#             raise NotImplementedError(regularizer_spec)

#     def forward(self, yhat, y):
#         return self.lossfn(yhat, y) + self.regularizer(yhat, y)

#     def __repr__(self):
#         return f'RegularizedLoss<{repr(self.lossfn)},{self.regularizer_spec}>'


# def get_dset_loaders_resultfactory(dset_spec:str) -> dict:
#     dct, class_names = match(dset_spec, DSETS)
#     if any(dset_spec.startswith(x) for x in {'intel_mobileodt:',
#                                              'chexpert_small_ID:'}):
#         #  dct['result_factory'] = lambda: TL.MultiLabelBinaryClassification(
#                 #  class_names, binarize_fn=lambda yh: (T.sigmoid(yh)>.5).long())
#         dct['result_factory'] = lambda: TL.MultiClassClassification(
#                 len(class_names), binarize_fn=lambda yh: yh.softmax(1).argmax(1))
#     elif any(dset_spec.startswith(x) for x in {'chexpert:', 'chexpert_small:'}):
#         dct['result_factory'] = lambda: CheXpertMultiLabelBinaryClassification(
#             class_names, binarize_fn=lambda yh: (yh.sigmoid()>.5).long(), report_avg=True)
#     else:
#         raise NotImplementedError(f"I don't know how to create the result factory for {dset_spec}")
#     return dct

# class CheXpertMultiLabelBinaryClassification(TL.MultiLabelBinaryClassification):
#     def update(self, yhat, y, loss) -> None:
#         self.num_samples += yhat.shape[0]
#         self.loss += loss.item()
#         assert yhat.shape == y.shape
#         assert yhat.ndim == 2 and yhat.shape[1] == len(self._cms), "sanity check: model outputs expected prediction shape"
#         binarized = self._binarize_fn(yhat)
#         assert binarized.dtype == T.long, 'sanity check binarize fn'
#         assert binarized.shape == y.shape, 'sanity check binarize fn'
#         ignore = (y != 2)  # ignore uncertainty labels
#         for i, (kls, cm) in enumerate(self._cms.items()):
#             rows = ignore[:, i]
#             if rows.sum() == 0:
#                 continue  # don't update a confusion matrix if all data for this class is ignored
#             cm += metrics.confusion_matrix(y[rows, i], binarized[rows, i], num_classes=2).cpu()


# def get_deepfix_train_strategy(args:'TrainOptions'):
#     deepfix_spec = args.deepfix
#     if deepfix_spec == 'off':
#         return TL.train_one_epoch
#     elif deepfix_spec.startswith('reinit:'):
#         _, N, P, R = deepfix_spec.split(':')
#         return dfs.DeepFix_TrainOneEpoch(int(N), float(P), int(R), TL.train_one_epoch)
#     elif deepfix_spec.startswith('dhist:'):
#         fp = deepfix_spec.split(':', 1)[1]
#         assert exists(fp), f'histogram file not found: {fp}'
#         return dfs.DeepFix_DHist(fp)
#     elif deepfix_spec.startswith('dfhist:'):
#         fp = deepfix_spec.split(':', 1)[1]
#         assert exists(fp), f'histogram file not found: {fp}'
#         return dfs.DeepFix_DHist(fp, fixed=True)
#     elif deepfix_spec == 'fixed':
#         return dfs.DeepFix_DHist('', fixed=True, init_with_hist=False)
#     elif deepfix_spec.startswith('beta:'):
#         alpha, beta = deepfix_spec.split(':')[1:]
#         return dfs.DeepFix_LambdaInit(
#             lambda cfg: init_from_beta(cfg.model, float(alpha), float(beta)))
#     elif deepfix_spec.startswith('ghaarconv2d:'):
#         ignore_layers = deepfix_spec.split(':')[1].split(',')
#         return dfs.DeepFix_LambdaInit(
#             lambda cfg: (
#                 print(f'initialize {deepfix_spec}'),
#                 convert_conv2d_to_gHaarConv2d(cfg.model, ignore_layers=ignore_layers),
#                 reset_optimizer(args.opt, cfg.model),
#                 print(cfg.model)
#             ))
#     else:
#         raise NotImplementedError(deepfix_spec)


# def train_config(args:'TrainOptions') -> TL.TrainConfig:
#     return TL.TrainConfig(
#         **get_model_opt_loss(
#             args.model, args.opt, args.lossfn, args.loss_reg, args.device),
#         **get_dset_loaders_resultfactory(args.dset),
#         device=args.device,
#         epochs=args.epochs,
#         start_epoch=args.start_epoch,
#         train_one_epoch=get_deepfix_train_strategy(args),
#         experiment_id=args.experiment_id,
#     )


# @dc.dataclass
# class TrainOptions:
#     """High-level configuration for training PyTorch models
#     on the IntelMobileODTCervical dataset.
#     """
#     epochs:int = 50
#     start_epoch:int = 0  # if "--start_epoch 1", then don't evaluate perf before training.
#     device:str = 'cuda' if T.cuda.is_available() else 'cpu'
#     dset:str = None #choice(
#         #  'intel_mobileodt:train:val:test:v1',
#         #  'intel_mobileodt:train+additional:val:test:v1',
#         #  'intel_mobileodt:train+additional:noval:test:v1',
#         #  'chexpert:.8:.2', 'chexpert:.01:.01', 'chexpert:.001:.001',
#         #  'chexpert_small:.8:.2', 'chexpert_small:.01:.01',
#         #   'chexpert_small:.001:.001',
#         #  default='intel_mobileodt:train:val:test:v1')
#     opt:str = 'SGD:lr=.001:momentum=.9:nesterov=1'
#     lossfn:str = None  # choices:
#         #  'BCEWithLogitsLoss',
#         #  'CrossEntropyLoss', 
#         #  'CE_intelmobileodt',
#         #  'chexpert_uignore', 
#         #  'chexpert_identity:N' for some N=num_identities predicted by model (compared to identities y%N)
#     loss_reg:str = 'none'  # Optionally add a regularizer to the loss.  loss + reg.  Accepted values:  'none', 'deepfixmlp:X' where X is a positive float denoting the lambda in l1 regularizer
#     model:str = 'resnet18:imagenet:3:3'  # Model specification adheres to the template "model_name:pretraining:in_ch:out_ch"
#     deepfix:str = 'off'  # DeepFix Re-initialization Method.
#                          #  "off" or "reinit:N:P:R" or "d[f]hist:path_to_histogram.pth"
#                          #  or "beta:A:B" for A,B as (float) parameters of the beta distribution
#                          # 'ghaarconv2d:layer1,layer2' Replaces all spatial convolutions with GHaarConv2d layer except the specified layers
#     experiment_id:str = os.environ.get('run_id', 'debugging')
#     prune:str = 'off'

#     def execute(self):
#         cfg = train_config(self)
#         cfg.train(cfg)


# def main():
#     p = ArgumentParser()
#     p.add_arguments(TrainOptions, dest='TrainOptions')

# #     args = p.parse_args(["--dset", "chexpert_small:.01:.01", "--opt", "Adam:lr=0.001", "--lossfn", "chexpert_uignore", "--model", "waveletmlp:300:1:14:7:1:1:2", "--loss_reg", "none"]).TrainOptions

#     print(args)
#     cfg = train_config(args)
    
# # python deepfix/train.py --dset chexpert_small:.01:.01 --opt Adam:lr=0.001 --lossfn chexpert_uignore --model waveletmlp:300:1:14:7:1:1:2 --loss_reg none    
    
#     if args.prune != 'off':
#         assert args.prune.startswith('ChannelPrune:')
#         raise NotImplementedError('code is a bit hardcoded, so it is not available without hacking on it.')
#         print(args.prune)
#         from explainfix import channelprune
#         from deepfix.weight_saliency import costfn_multiclass
#         a = sum([x.numel() for x in cfg.model.parameters()])
#         channelprune(cfg.model, pct=5, grad_cost_fn=costfn_multiclass,
#                      loader=cfg.train_loader, device=cfg.device, num_minibatches=10)
#         b = sum([x.numel() for x in cfg.model.parameters()])
#         assert a/b != 1
#         print(f'done channelpruning.  {a/b}')

#     cfg.train(cfg)
#     #  import IPython ; IPython.embed() ; import sys ; sys.exit()

#     #  with T.profiler.profile(
#     #      activities=[
#     #          T.profiler.ProfilerActivity.CPU,
#     #          T.profiler.ProfilerActivity.CUDA,
#     #      ], with_modules=True,
#     #  ) as p:
#     #      cfg.train(cfg)
#     #  print(p.key_averages().table(
#     #      sort_by="self_cuda_time_total", row_limit=-1))


# if __name__ == "__main__":
#     main()


TrainOptions(epochs=50, start_epoch=0, device='cuda', dset='chexpert_small:.01:.01', opt='Adam:lr=0.001', lossfn='chexpert_uignore', loss_reg='none', model='waveletmlp:300:1:14:7:1:1:2', deepfix='off', experiment_id='debugging', prune='off')
Checkpoint ./results/debugging/checkpoints/epoch_0.pth


100%|█████████████████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  5.56it/s]
INFO:simplepytorch.logging_tools:DataLogger writing to file: /ocean/projects/cie160013p/elvinj/DeepFix/deepfix/results/debugging/log/20220127T232131.085416_perf.csv


epoch:    0, seconds_training_epoch: nan
	TRAIN RESULTS: train_Loss:  109.22343, train_Num Samples:  2234.00000, train_MCC No Finding:  0.04027, train_MCC Enlarged Cardiomediastinum: -0.02440, train_MCC Cardiomegaly: -0.02490, train_MCC Lung Opacity: -0.04629, train_MCC Lung Lesion:  nan, train_MCC Edema: -0.13492, train_MCC Consolidation: -0.00033, train_MCC Pneumonia:  nan, train_MCC Atelectasis:  nan, train_MCC Pneumothorax:  0.01710, train_MCC Pleural Effusion:  0.01894, train_MCC Pleural Other: -0.04473, train_MCC Fracture:  nan, train_MCC Support Devices: -0.10193, train_MCC AVG:  nan, train_Precision No Finding:  0.33333, train_Precision Enlarged Cardiomediastinum:  0.05077, train_Precision Cardiomegaly:  0.00000, train_Precision Lung Opacity:  0.48478, train_Precision Lung Lesion:  0.03876, train_Precision Edema:  0.14870, train_Precision Consolidation:  0.06818, train_Precision Pneumonia:  0.02957, train_Precision Atelectasis:  0.00000, train_Precision Pneumothorax:  0.16667, 

100%|█████████████████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  5.41it/s]


epoch:    1, seconds_training_epoch: 23.8135
	TRAIN RESULTS: train_Loss:  63.05016, train_Num Samples:  2234.00000, train_MCC No Finding: -0.00706, train_MCC Enlarged Cardiomediastinum: -0.01602, train_MCC Cardiomegaly:  0.00602, train_MCC Lung Opacity:  0.03757, train_MCC Lung Lesion: -0.01657, train_MCC Edema:  0.00535, train_MCC Consolidation: -0.00611, train_MCC Pneumonia:  0.05242, train_MCC Atelectasis: -0.00055, train_MCC Pneumothorax:  0.03496, train_MCC Pleural Effusion:  0.04135, train_MCC Pleural Other: -0.00934, train_MCC Fracture:  0.06132, train_MCC Support Devices:  0.10248, train_MCC AVG:  0.02042, train_Precision No Finding:  0.00000, train_Precision Enlarged Cardiomediastinum:  0.00000, train_Precision Cardiomegaly:  0.15000, train_Precision Lung Opacity:  0.51369, train_Precision Lung Lesion:  0.00000, train_Precision Edema:  0.25676, train_Precision Consolidation:  0.00000, train_Precision Pneumonia:  0.13333, train_Precision Atelectasis:  0.17347, train_Precision P

...

100%|█████████████████████████████████████████████████████████████████████████████████| 16/16 [00:03<00:00,  5.11it/s]


epoch:   20, seconds_training_epoch: 25.1197
	TRAIN RESULTS: train_Loss:  51.51305, train_Num Samples:  2234.00000, train_MCC No Finding:  0.11310, train_MCC Enlarged Cardiomediastinum:  nan, train_MCC Cardiomegaly:  nan, train_MCC Lung Opacity:  0.21105, train_MCC Lung Lesion:  nan, train_MCC Edema:  0.14310, train_MCC Consolidation:  nan, train_MCC Pneumonia:  nan, train_MCC Atelectasis:  nan, train_MCC Pneumothorax:  nan, train_MCC Pleural Effusion:  0.21536, train_MCC Pleural Other:  nan, train_MCC Fracture:  nan, train_MCC Support Devices:  0.23835, train_MCC AVG:  nan, train_Precision No Finding:  0.40741, train_Precision Enlarged Cardiomediastinum:  0.00000, train_Precision Cardiomegaly:  0.00000, train_Precision Lung Opacity:  0.58955, train_Precision Lung Lesion:  0.00000, train_Precision Edema:  0.46061, train_Precision Consolidation:  0.00000, train_Precision Pneumonia:  0.00000, train_Precision Atelectasis:  0.00000, train_Precision Pneumothorax:  0.00000, train_Precision P

100%|█████████████████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  5.48it/s]


epoch:   21, seconds_training_epoch: 24.9122
	TRAIN RESULTS: train_Loss:  51.24277, train_Num Samples:  2234.00000, train_MCC No Finding:  0.14782, train_MCC Enlarged Cardiomediastinum:  nan, train_MCC Cardiomegaly:  nan, train_MCC Lung Opacity:  0.21655, train_MCC Lung Lesion:  nan, train_MCC Edema:  0.16698, train_MCC Consolidation:  nan, train_MCC Pneumonia:  nan, train_MCC Atelectasis:  nan, train_MCC Pneumothorax: -0.00623, train_MCC Pleural Effusion:  0.23422, train_MCC Pleural Other:  nan, train_MCC Fracture:  nan, train_MCC Support Devices:  0.26830, train_MCC AVG:  nan, train_Precision No Finding:  0.54545, train_Precision Enlarged Cardiomediastinum:  0.00000, train_Precision Cardiomegaly:  0.00000, train_Precision Lung Opacity:  0.59929, train_Precision Lung Lesion:  0.00000, train_Precision Edema:  0.51007, train_Precision Consolidation:  0.00000, train_Precision Pneumonia:  0.00000, train_Precision Atelectasis:  0.00000, train_Precision Pneumothorax:  0.00000, train_Precisi

 36%|████████████████████████████▍                                                   | 53/149 [00:09<00:15,  6.07it/s]

In [4]:
# os.getcwd()
# !ls

In [4]:
# args = {}
# args['dset'] = 'chexpert_small:.01:.01'
# args['opt'] = 'Adam:lr=0.001'
# args['lossfn'] = 'chexpert_uignore'
# args['model'] = 'waveletmlp:300:1:14:7:1:1:2'
# args['loss_reg'] = None
# args['prune'] = 'off'

In [9]:
# args.dset

In [21]:
# import pandas as pd

In [23]:
# t = pd.read_csv('../data/CheXpert-v1.0-small/train.csv')